# 从网页中截取文章内容

## 步骤1：参数设置

In [1]:
urls = """
http://blog.jobbole.com/105602/
http://www.cnblogs.com/qw12/p/6294430.html

http://www.cnblogs.com/neopenx/p/4453161.html
"""
url_list = urls.splitlines()
while '' in url_list:
    url_list.remove('')
    
url = url_list[0]
print(url)

http://blog.jobbole.com/105602/


## 步骤2：查找相应的截取关键字

In [2]:
if 'blog.jobbole.com' in url:  # 伯乐在线
    title_key = '.entry-header'
    content_key = '.entry'
    print('伯乐在线 http://blog.jobbole.com')
elif 'blog.csdn.net' in url:  # csdn
    title_key = '.link_title'
    content_key = '#article_content'
    print('csdn http://blog.csdn.net')
elif 'www.codingpy.com' in url:  # 编程派网址
    title_key = '.header h1'
    content_key = '.article-content'
    print('编程派网址 http://www.codingpy.com')
elif 'www.cnblogs.com' in url:  # 博客园
    title_key = '#cb_post_title_url'
    content_key = '#cnblogs_post_body'
    print('博客园 http://www.cnblogs.com')
else:
    title_key = '#keraspython'
    content_key = '.section'
    print('其它')

伯乐在线 http://blog.jobbole.com


## 步骤3：发出网页请求，接收响应

In [3]:
from urllib import request
req = request.Request(url)
res = request.urlopen(req)
html = res.read().decode('utf-8')
print(html)

<!DOCTYPE html>   
<!--[if lt IE 7 ]> <html lang="en" class="no-js ie6"> <![endif]-->
<!--[if IE 7 ]> <html lang="en" class="no-js ie7"> <![endif]-->
<!--[if IE 8 ]> <html lang="en" class="no-js ie8"> <![endif]-->
<!--[if IE 9 ]> <html lang="en" class="no-js ie9"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!--> <html lang="zh-CN" prefix="og: http://ogp.me/ns#" class="no-js"> <!--<![endif]-->


<!-- BEGIN head -->
<head>

	<!-- Title -->
	
	
	<!-- Meta Tags -->
	<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
	<meta name="viewport" content="width=device-width, initial-scale=1.0" />
	
	<!-- Favicon & Mobileicon -->
	<link rel="shortcut icon" href="http://jbcdn2.b0.upaiyun.com/2013/12/favicon.png" />
	<link rel="apple-touch-icon" href="http://jbcdn2.b0.upaiyun.com/2013/12/jobbole-blog-logo-mobile.png" />
	
	<!-- RSS, Atom & Pingbacks -->
		<link rel="alternate" title="文章 &#8211; 伯乐在线 RSS Feed" href="http://blog.jobbole.com/feed/" />
		<link rel="alternate" title="RSS .92

## 步骤4：从网页中提取文章标题和内容

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.select(title_key)[0].text.strip()  # 文章标题
print('文章标题：', title)
content = soup.select(content_key)[0]  # 文章内容

文章标题： TensorFlow深度学习，一篇文章就够了


## 步骤5：下载文章中的图片

### 5.1 哈希码生成函数，用于给图片重新命名

In [5]:
import hashlib
def md5(name):
    """ 将字符串转成哈希码 """
    if not isinstance(name, str):
        name = str(name)
    md5 = hashlib.md5()
    md5.update(name.encode('utf-8'))
    return md5.hexdigest()

### 5.2 下载图片，并修改文章图片的超链接

In [6]:
import re
import os
content = str(content)
pattern = '<img .*?src=\"(.*?)\"'
re_image = re.compile(pattern)
for image_link in re_image.findall(content):
    if not os.path.exists('images'):
        os.mkdir('images')
    filename = 'images/' + md5(image_link) + os.path.splitext(image_link)[-1]
    try:
        request.urlretrieve(image_link, filename)
        print('下载完成', filename)
    except Exception as e:
        print('图片出错', e)
    else:
        content = content.replace(image_link, filename)
print('已完成--------')

下载完成 images/3bfd4a249a13fddf184b9d375ea74564.jpg
下载完成 images/163fd797320aaea39de5b2e4d00eaba0.jpg
下载完成 images/f5d36cb31b98ffdf02ec09f8556e2fba.jpg
下载完成 images/4cf1c717943998a2ccac1b5798b64ab5.jpg
下载完成 images/b9c111b37c11e09aacf74e35db5d0f5a.jpg
下载完成 images/343d2df0f60dfa7973735f7698749bba.jpg
下载完成 images/dea398a41f059908f35e0dbc1ea91025.jpg
下载完成 images/180fd46b962251c1de86d93f985f68dd.jpg
下载完成 images/f1fb31a38a9aa1bbfd85cfb62578081e.jpg
下载完成 images/e18cb52dc30176d8e635ddf7a765ba21.jpg
下载完成 images/a04e5bbd8f2b51ffcb1a0ed542221cf4.jpg
下载完成 images/c8a313e8f3fd7d5c161ec0b05335c632.jpg
已完成--------


## 步骤6：将截取的文章标题和内容重新组合成新的网页文件

In [7]:
html_template = """<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="{origin}">原文链接</a></p>
<p><center><h1>{title}</h1></center></p>
    {content}
</body></html>"""
html = html_template.format(origin=url, title=title, content=content)
print(html)

<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="http://blog.jobbole.com/105602/">原文链接</a></p>
<p><center><h1>TensorFlow深度学习，一篇文章就够了</h1></center></p>
    <div class="entry">
<div class="textwidget"></div>
<div class="copyright-area">原文出处： <a href="http://www.52cs.org/?p=1157" target="_blank">我爱计算机 （@tobe迪豪 ）</a>   </div><p><img class="alignnone" height="345" src="images/3bfd4a249a13fddf184b9d375ea74564.jpg" width="690"/></p>
<p>作者: <a href="http://weibo.com/tobegit3hub">陈迪豪</a>，就职小米科技，深度学习工程师，TensorFlow代码提交者。</p>
<h3>TensorFlow深度学习框架</h3>
<p>Google不仅是大数据和云计算的领导者，在机器学习和深度学习上也有很好的实践和积累，在2015年年底开源了内部使用的深度学习框架<a href="https://github.com/tensorflow/tensorflow">TensorFlow</a>。</p>
<p>与Caffe、Theano、Torch、MXNet等框架相比，TensorFlow在Github上Fork数和Star数都是最多的，而且在图形分类、音频处理、推荐系统和自然语言处理等场景下都有丰富的应用。最近流行的Keras框架底层默认使用TensorFlow，著名的斯坦福CS231n课程使用TensorFlow作为授课和作业的编程语言，国内外多本TensorFlow书籍已经在筹备或者发售中，AlphaGo开发团队Deepmind也计划将神经网络应用迁移到TensorFlow中，这无不印证了TensorFlow在业界的流行程度。</p>
<p>TensorFlo

## 步骤7：将文件写入磁盘

In [8]:
filename = title + ".html"
with open(filename, "w") as f:
    f.write(html.replace(u'\xa0', u' ').replace(u'\U0001f60a', u' '))  # 在windows中出错，所以这里进行了字符串替换

## (完)